In [1]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:36355")
client

<Client: 'tcp://127.0.0.1:36355' processes=2 threads=2, memory=8.00 GiB>

In [2]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

import os
import pickle

import SAM
import float_data as flt

from sklearn.decomposition import PCA

import importlib
importlib.reload(SAM)
importlib.reload(flt)

from dask import delayed

In [3]:
# Number of classes 
n_classes = 7
model_folder = 'model'
data_folder = 'data'

ids = ['r1i1p1f2', 'r3i1p1f2', 'r2i1p1f2', 'r8i1p1f2', 'r4i1p1f2', 'r5i1p1f3', 'r7i1p1f3', 'r9i1p1f2', 'r6i1p1f3', 'r10i1p1f2']
ids = ['r6i1p1f3', 'r10i1p1f2']
mask = np.load('data/mask.npy', allow_pickle=True)
areas = {}

In [4]:
for m_id in ids:
    
    print('Starting {}'.format(m_id))
    path_id = '{}/{}'.format(model_folder, m_id)
    path_n = '{}/{}/{}'.format(model_folder, m_id, n_classes)
    path_n_data = '{}/{}/{}'.format(data_folder, m_id, n_classes)
    
    with open('{}/pca.obj'.format(path_id), 'rb') as file:
        pca = pickle.load(file)
        file.close()
        
    with open('{}/gmm.obj'.format(path_n), 'rb') as file:
        gmm = pickle.load(file)
        file.close()
    
    options = {'memberId' : m_id}
    data = flt.retrieve_profiles(timeRange = slice('1965-01', '1994-12'), mask=mask, options=options)
    data_sampled = flt.normalise_data(data, ('n', 'time'))
    data_trans = flt.pca_transform(data_sampled, pca)
    data_classes = SAM.BW_filter(flt.gmm_prob(data_trans, gmm), 4, 1/1.5, 'time', btype='high').argmax('k')
    #data_classes = flt.gmm_classify(data_trans, gmm)
    print('Finished setup for {}'.format(m_id))
    areas[m_id] = flt.count_area(data_classes, n_classes).compute()
    
    
    if not os.path.isdir(path_n_data):
        os.makedirs(path_n_data)
    with open('{}/areas_prob_high15.obj'.format(path_n_data), 'wb') as file:
        pickle.dump(areas[m_id], file)
        file.close()
    
print('Done!')

Starting r6i1p1f3


/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/core/indexing.py:1228: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/core/indexing.py:1228: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


Finished setup for r6i1p1f3
Starting r10i1p1f2


/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/core/indexing.py:1228: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/core/indexing.py:1228: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


Finished setup for r10i1p1f2
Done!
